In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from datetime import datetime
from dateutil import tz
from pathlib import Path
from neuroconv.datainterfaces import BlackrockRecordingInterface

In [4]:
file_path = "/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_BlackRock_withCognitiveEvents/BlackRock_raw_H0.2_071823001.ns6"

interface = BlackrockRecordingInterface(file_path=file_path, verbose=False)

# Extract what metadata we can from the source files
metadata = interface.get_metadata()

# # For data provenance we add the time zone information to the conversion
# session_start_time = metadata["NWBFile"]["session_start_time"]
# session_start_time = session_start_time.replace(tzinfo=tz.gettz("US/Pacific")).isoformat()
# metadata["NWBFile"].update(session_start_time=session_start_time)

# # Choose a path for saving the nwb file and run the conversion
# nwbfile_path = f"{path_to_save_nwbfile}"
# interface.run_conversion(nwbfile_path=nwbfile_path, metadata=metadata)

In [7]:
interface.recording_extractor

BlackrockRecordingExtractor: 32 channels - 30.0kHz - 1 segments - 444,255,989 samples 
                             14,808.53s (4.11 hours) - int16 dtype - 26.48 GiB
  file_path: /media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_BlackRock_withCognitiveEvents/BlackRock_raw_H0.2_071823001.ns6